In [69]:
import numpy
import pandas as pd
import os
import tempfile
import re
import csv

In [70]:
filePath = "\MP-Data"
videos = {}
dict_keyword = "session_"

In [71]:
#get all study-relevant details
for folder in os.listdir(filePath):
    newPath = os.path.join(filePath,folder)
    if (os.path.isdir(newPath)):
        for fileName in os.listdir(newPath):
            if (fileName == "cam_output.mp4"):
                videos[dict_keyword+folder] = os.path.join(newPath, fileName)

In [72]:
videos

{'session_1': '\\MP-Data\\1\\cam_output.mp4',
 'session_10': '\\MP-Data\\10\\cam_output.mp4',
 'session_11': '\\MP-Data\\11\\cam_output.mp4',
 'session_12': '\\MP-Data\\12\\cam_output.mp4',
 'session_13': '\\MP-Data\\13\\cam_output.mp4',
 'session_14': '\\MP-Data\\14\\cam_output.mp4',
 'session_15': '\\MP-Data\\15\\cam_output.mp4',
 'session_16': '\\MP-Data\\16\\cam_output.mp4',
 'session_17': '\\MP-Data\\17\\cam_output.mp4',
 'session_18': '\\MP-Data\\18\\cam_output.mp4',
 'session_19': '\\MP-Data\\19\\cam_output.mp4',
 'session_2': '\\MP-Data\\2\\cam_output.mp4',
 'session_20': '\\MP-Data\\20\\cam_output.mp4',
 'session_21': '\\MP-Data\\21\\cam_output.mp4',
 'session_22': '\\MP-Data\\22\\cam_output.mp4',
 'session_23': '\\MP-Data\\23\\cam_output.mp4',
 'session_24': '\\MP-Data\\24\\cam_output.mp4',
 'session_25': '\\MP-Data\\25\\cam_output.mp4',
 'session_26': '\\MP-Data\\26\\cam_output.mp4',
 'session_27': '\\MP-Data\\27\\cam_output.mp4',
 'session_3': '\\MP-Data\\3\\cam_output.mp4'

In [73]:
def getVideoDetails(filepath):
    tmpf = "metadata.txt"
    os.system("ffmpeg -i \"%s\" 2> %s" % (filepath, tmpf))
    metadata = {}
    with open(tmpf) as f:
        lines = f.readlines()
    for l in lines:
        l = l.strip()
        if l.startswith('Duration'):
            metadata['duration'] = re.search('Duration: (.*?),', l).group(0).split(':',1)[1].strip(' ,')
            metadata['bitrate'] = re.search("bitrate: (\d+ kb/s)", l).group(0).split(':')[1].strip()
        if l.startswith('Stream #0:0'):
            metadata['video'] = {}
            metadata['video']['codec'], metadata['video']['profile'] = \
                [e.strip(' ,()') for e in re.search('Video: (.*? \(.*?\)),? ', l).group(0).split(':')[1].split('(')]
            metadata['video']['resolution'] = re.search('([1-9]\d+x\d+)', l).group(1)
            metadata['video']['bitrate'] = re.search('(\d+ kb/s)', l).group(1)
            metadata['video']['fps'] = re.search('(\d+ fps)', l).group(1)
        if l.startswith('Stream #0:1'):
            metadata['audio'] = {}
            metadata['audio']['codec'] = re.search('Audio: (.*?) ', l).group(1)
            metadata['audio']['frequency'] = re.search(', (.*? Hz),', l).group(1)
            metadata['audio']['bitrate'] = re.search(', (\d+ kb/s)', l).group(1)
    return metadata


In [74]:
#get metadata for each video
metadataCompiled = {}
for v in videos:
    result = getVideoDetails(videos[v])
    metadataCompiled[v] = result
print (metadataCompiled)

{'session_1': {'duration': '00:21:21.60', 'bitrate': '1266 kb/s', 'video': {'codec': 'h264', 'profile': 'High', 'resolution': '2560x1440', 'bitrate': '1130 kb/s', 'fps': '30 fps'}, 'audio': {'codec': 'aac', 'frequency': '44100 Hz', 'bitrate': '128 kb/s'}}, 'session_10': {'duration': '00:16:12.90', 'bitrate': '1433 kb/s', 'video': {'codec': 'h264', 'profile': 'High', 'resolution': '2560x1440', 'bitrate': '1297 kb/s', 'fps': '30 fps'}, 'audio': {'codec': 'aac', 'frequency': '44100 Hz', 'bitrate': '128 kb/s'}}, 'session_11': {'duration': '00:18:31.70', 'bitrate': '1081 kb/s', 'video': {'codec': 'h264', 'profile': 'High', 'resolution': '2560x1440', 'bitrate': '945 kb/s', 'fps': '30 fps'}, 'audio': {'codec': 'aac', 'frequency': '44100 Hz', 'bitrate': '128 kb/s'}}, 'session_12': {'duration': '00:26:13.27', 'bitrate': '1336 kb/s', 'video': {'codec': 'h264', 'profile': 'High', 'resolution': '2560x1440', 'bitrate': '1200 kb/s', 'fps': '30 fps'}, 'audio': {'codec': 'aac', 'frequency': '44100 Hz'

In [93]:
#compile into 2x2 matrix to make it easier to parse
    #Kartik sends his regards for those trying to understand what's going on
simpleForCsv = {}
for base in metadataCompiled:
    simpleForCsv[base] = {}
    for typeName in metadataCompiled[base]:
        for typeFactor in metadataCompiled[base][typeName]:
            if (isinstance(metadataCompiled[base][typeName],dict)):
                combinedName = typeName+":"+typeFactor
                simpleForCsv[base][combinedName] = metadataCompiled[base][typeName][typeFactor]
            else:
                simpleForCsv[base][typeName] = metadataCompiled[base][typeName]

In [94]:
simpleForCsv

{'session_1': {'duration': '00:21:21.60',
  'bitrate': '1266 kb/s',
  'video:codec': 'h264',
  'video:profile': 'High',
  'video:resolution': '2560x1440',
  'video:bitrate': '1130 kb/s',
  'video:fps': '30 fps',
  'audio:codec': 'aac',
  'audio:frequency': '44100 Hz',
  'audio:bitrate': '128 kb/s'},
 'session_10': {'duration': '00:16:12.90',
  'bitrate': '1433 kb/s',
  'video:codec': 'h264',
  'video:profile': 'High',
  'video:resolution': '2560x1440',
  'video:bitrate': '1297 kb/s',
  'video:fps': '30 fps',
  'audio:codec': 'aac',
  'audio:frequency': '44100 Hz',
  'audio:bitrate': '128 kb/s'},
 'session_11': {'duration': '00:18:31.70',
  'bitrate': '1081 kb/s',
  'video:codec': 'h264',
  'video:profile': 'High',
  'video:resolution': '2560x1440',
  'video:bitrate': '945 kb/s',
  'video:fps': '30 fps',
  'audio:codec': 'aac',
  'audio:frequency': '44100 Hz',
  'audio:bitrate': '128 kb/s'},
 'session_12': {'duration': '00:26:13.27',
  'bitrate': '1336 kb/s',
  'video:codec': 'h264',
  

In [95]:
master = pd.DataFrame.from_dict(simpleForCsv)
master

,session_1,session_10,session_11,session_12,session_13,session_14,session_15,session_16,session_17,session_18,...,session_24,session_25,session_26,session_27,session_3,session_5,session_6,session_7,session_8,session_9
audio:bitrate,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,...,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s,128 kb/s
audio:codec,aac,aac,aac,aac,aac,aac,aac,aac,aac,aac,...,aac,aac,aac,aac,aac,aac,aac,aac,aac,aac
audio:frequency,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,...,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz,44100 Hz
bitrate,1266 kb/s,1433 kb/s,1081 kb/s,1336 kb/s,952 kb/s,1071 kb/s,1032 kb/s,1322 kb/s,1047 kb/s,1331 kb/s,...,1038 kb/s,1243 kb/s,1016 kb/s,1109 kb/s,1159 kb/s,892 kb/s,1058 kb/s,1119 kb/s,1048 kb/s,986 kb/s
duration,00:21:21.60,00:16:12.90,00:18:31.70,00:26:13.27,00:20:43.07,00:18:53.60,00:18:29.00,00:26:25.80,00:25:45.53,00:30:11.80,...,00:26:01.43,00:18:03.90,00:20:28.73,00:23:25.17,00:22:48.17,00:24:15.57,00:24:23.43,00:25:31.10,00:22:44.23,00:17:45.40
video:bitrate,1130 kb/s,1297 kb/s,945 kb/s,1200 kb/s,816 kb/s,936 kb/s,896 kb/s,1186 kb/s,911 kb/s,1194 kb/s,...,903 kb/s,1108 kb/s,880 kb/s,973 kb/s,1024 kb/s,756 kb/s,923 kb/s,983 kb/s,912 kb/s,850 kb/s
video:codec,h264,h264,h264,h264,h264,h264,h264,h264,h264,h264,...,h264,h264,h264,h264,h264,h264,h264,h264,h264,h264
video:fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,...,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps,30 fps
video:profile,High,High,High,High,High,High,High,High,High,High,...,High,High,High,High,High,High,High,High,High,High
video:resolution,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,...,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440,2560x1440


In [96]:
master.to_csv('video_metadata', index=False)

SyntaxError: EOL while scanning string literal (<ipython-input-96-130da7fb8e7b>, line 1)